In [1]:
# Import Our Packages to work with the data!
import pandas as pd
import os
import zipfile

working_directory = os.getcwd()

In [2]:
path = working_directory + '/yellow_tripdata_2023-08.parquet'
data = pd.read_parquet(path)

In [3]:
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-08-01 00:26:44,2023-08-01 00:45:25,1.0,4.30,1.0,N,263,90,1,21.9,3.50,0.5,5.35,0.00,1.0,32.25,2.5,0.00
1,1,2023-08-01 00:55:42,2023-08-01 01:00:53,1.0,0.00,1.0,N,132,132,3,5.1,2.75,0.5,0.00,0.00,1.0,9.35,0.0,1.75
2,2,2023-08-01 00:32:04,2023-08-01 01:09:03,1.0,20.61,2.0,N,132,13,1,70.0,0.00,0.5,14.80,0.00,1.0,90.55,2.5,1.75
3,2,2023-08-01 00:13:37,2023-08-01 00:41:15,1.0,13.37,1.0,N,138,254,2,54.1,6.00,0.5,0.00,6.55,1.0,69.90,0.0,1.75
4,1,2023-08-01 00:24:46,2023-08-01 00:41:31,2.0,3.20,5.0,N,140,143,4,20.0,0.00,0.0,0.00,0.00,1.0,21.00,0.0,0.00


In [4]:
# Checking datatypes for each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824209 entries, 0 to 2824208
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [5]:
data.shape

(2824209, 19)

In [6]:
with zipfile.ZipFile("taxi_zones.zip","r") as zip_ref:
    zip_ref.extractall("./shape")

In [7]:
import geopandas as gpd

In [8]:
# set the filepath and load in a shapefile 
fp = "shape/taxi_zones.shp"
map_data = gpd.read_file(fp)
# check the GeoDataframe
map_data.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."


In [9]:
map_data['centroid'] = map_data['geometry'].centroid.to_crs(epsg=4326) # Converts the shapes of the Pickup Zones into coordinates.

# Assign coordinates to each Location ID ,based on their centroids.
map_data['lon'] = map_data['centroid'].x
map_data['lat'] = map_data['centroid'].y
map_data.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,centroid,lon,lat
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19...",POINT (-74.17400 40.69183),-74.174002,40.691830
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343...",POINT (-73.83130 40.61675),-73.831300,40.616746
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ...",POINT (-73.84742 40.86447),-73.847422,40.864474
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20...",POINT (-73.97697 40.72375),-73.976968,40.723752
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14...",POINT (-74.18848 40.55266),-74.188485,40.552659


In [10]:
trips_data = data.merge(map_data,
                  left_on='PULocationID',
                  right_on='LocationID')

trips_data = trips_data.drop(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'LocationID', 'geometry', 'centroid'], axis=1)
trips_data = trips_data.rename(columns={"zone":"PU_zone", "borough":"PU_borough", "lon":"PU_lon", "lat":"PU_lat"})

trips_data = trips_data.merge(map_data,
                      left_on='DOLocationID',
                      right_on='LocationID')

trips_data = trips_data.drop(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'LocationID', 'geometry', 'centroid'], axis=1)
trips_data = trips_data.rename(columns={"zone":"DO_zone", "borough":"DO_borough", "lon":"DO_lon", "lat":"DO_lat"})

In [11]:
# Creating Primary Key
main_data = trips_data.drop_duplicates().reset_index(drop=True)
main_data['trip_id'] = trips_data.index
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2775235 entries, 0 to 2775234
Data columns (total 28 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [12]:
# Saving dataframe to be analyced with AWS Glue
main_data.to_parquet('data.parquet', engine='pyarrow')